# Sentiment Analysis

Think for this exploration I will primarily be drawing from _Opinion Digger: An Unsupervised Opinion Miner from Unstructured Product Reviews_.

### Load the Data

In [1]:
import pandas as pd

def load_data():
    article_table = pd.read_csv("../data/raw/kaggle1/articles1.csv")
    return article_table

In [2]:
articles = load_data()

### Load the Aspects

In [2]:
import json

aspect_data = {}
with open("../data/cache/kaggle_aspects.json") as in_file:
    aspect_data = json.load(in_file)

In [3]:
aspect_data

{'republicans': {'count': 80,
  'sentences': [0,
   2,
   3,
   4,
   6,
   10,
   12,
   18,
   19,
   21,
   23,
   1654,
   1657,
   1696,
   1698,
   1703,
   1712,
   1713,
   1715,
   1718,
   1720,
   1721,
   1728,
   1731,
   1733,
   1746,
   2374,
   2376,
   2383,
   2388,
   2413,
   2426,
   2429,
   2487,
   2494,
   2495,
   2495,
   2507,
   2518,
   2519,
   2520,
   2527,
   2534,
   2547,
   2559,
   2617,
   3271,
   3272,
   3272,
   3274,
   3281,
   3283,
   3291,
   3301,
   3302,
   3303,
   3314,
   3346,
   3353,
   3383,
   3383,
   3387,
   3388,
   3389,
   3389,
   3399,
   3408,
   4512,
   4948,
   5030,
   5031,
   5394,
   5405,
   5416,
   5428,
   5440,
   5478,
   5482,
   5483,
   5522],
  'pos': [['republicans', 'NNS']],
  'flr': 992.7738916792686},
 'fear': {'count': 14,
  'sentences': [0,
   55,
   86,
   1836,
   2704,
   2712,
   2915,
   3591,
   3646,
   4019,
   4972,
   5113,
   5535,
   5536],
  'pos': [['fear', 'NN']],
  'flr': 68.5857